In [1]:
# The code was removed by Watson Studio for sharing.

Company X has two plants, Plant1 and Plant2, which can produce Product1 and Product2

The cost of producing Product1 in Plant1 and Plant2 are 2 and 3 dollars, respectively

The cost of producing Product2 in Plant1 and Plant2 are 4 and 5 dollars, respectively

There are two customers, Customer1 and Customer2

Customer1 needs 1000 of Product1 and 1300 of Product2

Customer2 needs 1500 of Product1 and 1400 of Product2

Cost of transportation of Product1 from Plant1 to Customer1 and Customer2 is 0.2 and 0.3 dollars.

Cost of transportation of Product2 from Plant2 to Customer1 and Customer2 is 0.5 and 0.4 dollars.

What is the best production and transportation plan that minimizes the costs?

In [2]:
!pip install dse_do_utils

In [3]:
from dse_do_utils import DataManager, OptimizationEngine,ScenarioManager
from docplex.mp.model import Model
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

from docplex import __version__
__version__

'2.22.213'

In [4]:
class CplexSum():
    """Function class that adds a series of dvars into a cplex sum expression.
    To be used as a custom aggregation in a groupby.
    Usage:
        df2 = df1.groupby(['a']).agg({'xDVar':CplexSum(engine.mdl)}).rename(columns={'xDVar':'expr'})

    Sums the dvars in the 'xDVar' column into an expression
    """
    def __init__(self, mdl):
        self.mdl = mdl
    def __call__(self, dvar_series):
        return self.mdl.sum(dvar_series)


def extract_solution(df, extract_dvar_names=None, drop_column_names=None, drop:bool=True):
    df = df.copy()
    """Generalized routine to extract a solution value. 
    Can remove the dvar column from the df to be able to have a clean df for export into scenario."""
    if extract_dvar_names is not None:
        for xDVarName in extract_dvar_names:
            if xDVarName in df.columns:
                df[f'{xDVarName}_Solution'] = [dvar.solution_value for dvar in df[xDVarName]]
                if drop:
                    df = df.drop([xDVarName], axis = 1)
    if drop and drop_column_names is not None:
        for column in drop_column_names:
            if column in df.columns:
                df = df.drop([column], axis = 1)
    return df  


In [5]:
MODEL_NAME = 'SupplyChain'
SCENARIO_NAME = 'Base_Scenario' 

sm = ScenarioManager(model_name=MODEL_NAME, scenario_name=SCENARIO_NAME, project = project)
InputTables, OutputTables = sm.load_data_from_scenario()
print ( InputTables.keys())

print (  OutputTables.keys())

dict_keys(['Plants', 'Customers', 'Products', 'ProductionCosts', 'Demand', 'TransportationCosts', 'ProductionCapacity'])
dict_keys(['Delivery', 'TransportationPlan', 'ProductionPlan'])


# Modify Demand table

Define Data Models

In [6]:
Demand = pd.DataFrame([["Customer1","Product1",1000],
                       ["Customer1","Product2",1300],
                       ["Customer2","Product1",1500],
                       ["Customer2","Product2",1400]], columns = ["CustomerName","ProductName","Demand"])

InputTables['Demand']=Demand

OutputTables={}

## Save the inputs to a new scenario

In [7]:
MODEL_NAME = 'SupplyChain'
SCENARIO_NAME = 'High_Demand' 

In [8]:
sm = ScenarioManager(model_name=MODEL_NAME, scenario_name=SCENARIO_NAME, project =project )
sm.inputs=InputTables

sm.write_data_into_scenario_s(model_name=MODEL_NAME, scenario_name=SCENARIO_NAME, inputs=InputTables, outputs=OutputTables)

## Specify the primary key, convert primary key column to index

In [11]:
Plants = InputTables['Plants'].set_index(["PlantName"], verify_integrity=True)

Customers = InputTables['Customers'].set_index(["CustomerName"], verify_integrity=True)

Products = InputTables['Products'].set_index(["ProductName"], verify_integrity=True)

ProductionCosts= InputTables['ProductionCosts'].set_index(["PlantName","ProductName"], verify_integrity=True)

Demand= InputTables['Demand'].set_index(["CustomerName","ProductName"], verify_integrity=True)

TransportationCosts= InputTables['TransportationCosts'].set_index(["PlantName","CustomerName","ProductName"], verify_integrity=True)

ProductionCapacity= InputTables['ProductionCapacity'].set_index(["PlantName","ProductName"], verify_integrity=True)

## Build Optimization

In [12]:

mdl = Model(MODEL_NAME)

In [13]:
for i in InputTables.values():
    display(i)

,PlantName
0,Plant1
1,Plant2


,CustomerName
0,Customer1
1,Customer2


,ProductName
0,Product1
1,Product2


,PlantName,ProductName,ProductionCost
0,Plant1,Product1,2
1,Plant2,Product1,3
2,Plant1,Product2,4
3,Plant2,Product2,5


,CustomerName,ProductName,Demand
0,Customer1,Product1,1000
1,Customer1,Product2,1300
2,Customer2,Product1,1500
3,Customer2,Product2,1400


,PlantName,CustomerName,ProductName,TransportationCost
0,Plant1,Customer1,Product1,0.20
1,Plant1,Customer1,Product2,0.50
2,Plant1,Customer2,Product1,0.30
3,Plant1,Customer2,Product2,0.40
4,Plant2,Customer1,Product1,0.25
5,Plant2,Customer1,Product2,0.20
6,Plant2,Customer2,Product1,0.35
7,Plant2,Customer2,Product2,0.30


,PlantName,ProductName,Capacity
0,Plant1,Product1,500
1,Plant2,Product1,400
2,Plant1,Product2,300
3,Plant2,Product2,600


## Define Decision Variables

What are the production amount of Product1 and Product2 in either Plant1 or Plant2

How much of Product1 or Product2 should be sent from Plant1 and Plant2 to Customer1 and Customer2 

In [14]:
def continuous_var_series(df, mdl,**kargs):
    return pd.Series(mdl.continuous_var_list(df.index, **kargs), index = df.index)

In [15]:
ProductionPlan = pd.merge(Products.reset_index(drop=False),Plants.reset_index(drop=False),how="cross").set_index(["ProductName","PlantName"])
ProductionPlan["X_production"]= continuous_var_series(ProductionPlan, mdl, name="X_production", lb = 0)
ProductionPlan

X_production
ProductName PlantName                              
Product1    Plant1     X_production_Product1_Plant1
            Plant2     X_production_Product1_Plant2
Product2    Plant1     X_production_Product2_Plant1
            Plant2     X_production_Product2_Plant2

In [16]:
TransportationPlan = pd.merge(Products.reset_index(drop=False),Plants.reset_index(drop=False),how="cross").merge(Customers.reset_index(drop=False),how="cross").set_index(["ProductName","PlantName","CustomerName"])
TransportationPlan["X_transportation"]= continuous_var_series(TransportationPlan, mdl, name="X_transportation", lb = 0)
TransportationPlan

X_transportation
ProductName PlantName CustomerName                                            
Product1    Plant1    Customer1     X_transportation_Product1_Plant1_Customer1
                      Customer2     X_transportation_Product1_Plant1_Customer2
            Plant2    Customer1     X_transportation_Product1_Plant2_Customer1
                      Customer2     X_transportation_Product1_Plant2_Customer2
Product2    Plant1    Customer1     X_transportation_Product2_Plant1_Customer1
                      Customer2     X_transportation_Product2_Plant1_Customer2
            Plant2    Customer1     X_transportation_Product2_Plant2_Customer1
                      Customer2     X_transportation_Product2_Plant2_Customer2

## Define Constraints

### We can't generate more than the plant capacity

In [17]:
Plant_Product_Capacity = pd.merge(ProductionPlan.reset_index(drop=False), ProductionCapacity, on = ["ProductName","PlantName"])
Plant_Product_Capacity

,ProductName,PlantName,X_production,Capacity
0,Product1,Plant1,X_production_Product1_Plant1,500
1,Product1,Plant2,X_production_Product1_Plant2,400
2,Product2,Plant1,X_production_Product2_Plant1,300
3,Product2,Plant2,X_production_Product2_Plant2,600


In [18]:
for row in Plant_Product_Capacity.itertuples():
    mdl.add_constraint(row.X_production <= row.Capacity)

In [19]:
mdl.get_constraint_by_index(3)

docplex.mp.LinearConstraint[](X_production_Product2_Plant2,LE,600)

###  We must deliver at least the value of demand

In [20]:
Delivery = TransportationPlan.reset_index(drop = False)
Delivery = Delivery[["ProductName","CustomerName","X_transportation"]].groupby(["ProductName","CustomerName"]).agg(CplexSum(mdl)).reset_index(drop = False).rename(columns = {"X_transportation":"Total_Delivered"})   

In [21]:
Delivery = pd.merge(Delivery, Demand.reset_index(drop = False), on = ["ProductName","CustomerName"] )
Delivery

,ProductName,CustomerName,Total_Delivered,Demand
0,Product1,Customer1,X_transportation_Product1_Plant1_Customer1+X_transportation_Product1_Plant2_Customer1,1000
1,Product1,Customer2,X_transportation_Product1_Plant1_Customer2+X_transportation_Product1_Plant2_Customer2,1500
2,Product2,Customer1,X_transportation_Product2_Plant1_Customer1+X_transportation_Product2_Plant2_Customer1,1300
3,Product2,Customer2,X_transportation_Product2_Plant1_Customer2+X_transportation_Product2_Plant2_Customer2,1400


In [22]:
for row in Delivery.itertuples():
    mdl.add_constraint(row.Total_Delivered >= row.Demand)

### We cannot ship more than what we produced in a plant

In [23]:
Shipment = TransportationPlan.reset_index(drop = False)
Shipment = Shipment[["ProductName","PlantName","X_transportation"]].groupby(["ProductName","PlantName"]).agg(CplexSum(mdl)).reset_index(drop = False).rename(columns = {"X_transportation":"Total_Shipped"})
Shipment

,ProductName,PlantName,Total_Shipped
0,Product1,Plant1,X_transportation_Product1_Plant1_Customer1+X_transportation_Product1_Plant1_Customer2
1,Product1,Plant2,X_transportation_Product1_Plant2_Customer1+X_transportation_Product1_Plant2_Customer2
2,Product2,Plant1,X_transportation_Product2_Plant1_Customer1+X_transportation_Product2_Plant1_Customer2
3,Product2,Plant2,X_transportation_Product2_Plant2_Customer1+X_transportation_Product2_Plant2_Customer2


In [24]:
Plant_Shipment = pd.merge(ProductionPlan.reset_index(drop= False), Shipment, on =[ "ProductName","PlantName"])
Plant_Shipment

,ProductName,PlantName,X_production,Total_Shipped
0,Product1,Plant1,X_production_Product1_Plant1,X_transportation_Product1_Plant1_Customer1+X_transportation_Product1_Plant1_Customer2
1,Product1,Plant2,X_production_Product1_Plant2,X_transportation_Product1_Plant2_Customer1+X_transportation_Product1_Plant2_Customer2
2,Product2,Plant1,X_production_Product2_Plant1,X_transportation_Product2_Plant1_Customer1+X_transportation_Product2_Plant1_Customer2
3,Product2,Plant2,X_production_Product2_Plant2,X_transportation_Product2_Plant2_Customer1+X_transportation_Product2_Plant2_Customer2


In [25]:
for row in Plant_Shipment.itertuples():
    mdl.add_constraint(row.Total_Shipped <= row.X_production)

# KPIs and Objective Function

## Production Cost

In [26]:
TotalProductionCosts = pd.merge(ProductionPlan.reset_index(drop = False), ProductionCosts.reset_index(drop = False), on = ["ProductName","PlantName"])
display(TotalProductionCosts)

Total_Production_Costs = mdl.sum(TotalProductionCosts.ProductionCost * TotalProductionCosts.X_production)
mdl.add_kpi(Total_Production_Costs   , "Total_Production_Costs")

,ProductName,PlantName,X_production,ProductionCost
0,Product1,Plant1,X_production_Product1_Plant1,2
1,Product1,Plant2,X_production_Product1_Plant2,3
2,Product2,Plant1,X_production_Product2_Plant1,4
3,Product2,Plant2,X_production_Product2_Plant2,5


DecisionKPI(name=Total_Production_Costs,expr=2X_production_Product1_Plant1+3X_production_Product1_Plant2+4X_p..)

In [27]:
TotalTransportationCosts = pd.merge(TransportationPlan.reset_index(drop = False), TransportationCosts.reset_index(drop = False), on = ["ProductName","CustomerName","PlantName"])
display(TotalTransportationCosts)

Total_Transportation_Costs = mdl.sum(TotalTransportationCosts.TransportationCost * TotalTransportationCosts.X_transportation)
mdl.add_kpi(Total_Transportation_Costs   , "Total_Transportation_Costs")

,ProductName,PlantName,CustomerName,X_transportation,TransportationCost
0,Product1,Plant1,Customer1,X_transportation_Product1_Plant1_Customer1,0.20
1,Product1,Plant1,Customer2,X_transportation_Product1_Plant1_Customer2,0.30
2,Product1,Plant2,Customer1,X_transportation_Product1_Plant2_Customer1,0.25
3,Product1,Plant2,Customer2,X_transportation_Product1_Plant2_Customer2,0.35
4,Product2,Plant1,Customer1,X_transportation_Product2_Plant1_Customer1,0.50
5,Product2,Plant1,Customer2,X_transportation_Product2_Plant1_Customer2,0.40
6,Product2,Plant2,Customer1,X_transportation_Product2_Plant2_Customer1,0.20
7,Product2,Plant2,Customer2,X_transportation_Product2_Plant2_Customer2,0.30


DecisionKPI(name=Total_Transportation_Costs,expr=0.200X_transportation_Product1_Plant1_Customer1+0.300X_transport..)

In [28]:
Total_Costs = Total_Transportation_Costs + Total_Production_Costs
mdl.add_kpi(Total_Costs   , "Total_Costs")

DecisionKPI(name=Total_Costs,expr=2X_production_Product1_Plant1+3X_production_Product1_Plant2+4X_p..)

In [29]:
mdl.print_information()

Model: SupplyChain
 - number of variables: 12
   - binary=0, integer=0, continuous=12
 - number of constraints: 12
   - linear=12
 - parameters: defaults
 - objective: none
 - problem type is: LP


# Solve

In [30]:
mdl.minimize(Total_Costs)
mdl.solve()

In [31]:
mdl.report()

# Postprocessing

In [32]:
ProductionPlan = extract_solution(Plant_Product_Capacity, extract_dvar_names= ['X_production'] ,drop=False)
ProductionPlan

DOcplexException: Model<SupplyChain> did not solve successfully

In [49]:
TransportationPlan = extract_solution(TransportationPlan, extract_dvar_names= ['X_transportation'] ,drop=False)
TransportationPlan

X_transportation  \
ProductName PlantName CustomerName                                               
Product1    Plant1    Customer1     X_transportation_Product1_Plant1_Customer1   
                      Customer2     X_transportation_Product1_Plant1_Customer2   
            Plant2    Customer1     X_transportation_Product1_Plant2_Customer1   
                      Customer2     X_transportation_Product1_Plant2_Customer2   
Product2    Plant1    Customer1     X_transportation_Product2_Plant1_Customer1   
                      Customer2     X_transportation_Product2_Plant1_Customer2   
            Plant2    Customer1     X_transportation_Product2_Plant2_Customer1   
                      Customer2     X_transportation_Product2_Plant2_Customer2   

                                    X_transportation_Solution  
ProductName PlantName CustomerName                             
Product1    Plant1    Customer1                           0.0  
                      Customer2                         500.0  
            Plant2    Customer1                         200.0  
                      Customer2                           0.0  
Product2    Plant1    Customer1                           0.0  
                      Customer2                         300.0  
            Plant2    Customer1                         300.0  
                      Customer2                         100.0

In [45]:
Delivery = extract_solution(Delivery, extract_dvar_names= ['Total_Delivered'] ,drop=False)

Delivery

,ProductName,CustomerName,Total_Delivered,Demand,Total_Delivered_Solution
0,Product1,Customer1,X_transportation_Product1_Plant1_Customer1+X_transportation_Product1_Plant2_Customer1,200,200.0
1,Product1,Customer2,X_transportation_Product1_Plant1_Customer2+X_transportation_Product1_Plant2_Customer2,500,500.0
2,Product2,Customer1,X_transportation_Product2_Plant1_Customer1+X_transportation_Product2_Plant2_Customer1,300,300.0
3,Product2,Customer2,X_transportation_Product2_Plant1_Customer2+X_transportation_Product2_Plant2_Customer2,400,400.0


## save the data into watson studio optimization GUI

In [56]:
InputTables={}

InputTables['Plants']=Plants
InputTables['Customers']=Customers
InputTables['Products']=Products
InputTables['ProductionCosts']=ProductionCosts
InputTables['Demand']=Demand
InputTables['TransportationCosts']=TransportationCosts
InputTables['ProductionCapacity']=ProductionCapacity

sm = ScenarioManager(model_name=MODEL_NAME, scenario_name=SCENARIO_NAME, project =project )
sm.inputs=InputTables

In [57]:
OutputTables={}
OutputTables['Delivery']=Delivery
OutputTables['TransportationPlan']=TransportationPlan
OutputTables['ProductionPlan']=ProductionPlan
sm.outputs=OutputTables

In [58]:
sm.write_data_into_scenario_s(model_name=MODEL_NAME, scenario_name=SCENARIO_NAME, inputs=InputTables, outputs=OutputTables)